In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt

import math

In [8]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = torch.tensor(emb_size)

    def forward(self, tokens):
        return self.embedding(tokens.long()) * torch.sqrt(self.emb_size)


class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [9]:
class Head(nn.Module):
    def __init__(self, embed_size=512, head_size=8, sequence_length=500, dropout=0, device='cpu'):
        super().__init__()
        self.key_linear = nn.Linear(embed_size, head_size, bias=False)
        self.query_linear = nn.Linear(embed_size, head_size, bias=False)
        self.value_linear = nn.Linear(embed_size, head_size, bias=False)
        
        self.mask = torch.tril(torch.ones(sequence_length, sequence_length))

        self.dropout = nn.Dropout(dropout)
        self.device = device

    def forward(self, x):
        # B = Batch Size, T = Sequence Length, C = Channel / Embedding Size 
        B,T,C = x.shape
        key = self.key_linear(x)        # (B, T, C)
        query = self.query_linear(x)    # (B, T, C)
        value = self.value_linear(x)    # (B, T, C)
        
        score = query @ key.transpose(1, 2) * C ** -0.5                     # (B, T, C) @ (B, C, T) -> (B, T, T)
        mask = (self.mask[:T, :T] == 0).to(self.device)
        score = score.masked_fill(mask, float('-inf'))    # (B, T, T)
        score = F.softmax(score, dim=-1)                                    # (B, T, T)
        # score = self.dropout(score)
        out = score @ value             # (B, T, T) @ (B, T, C) -> (B, T, C)
        out = self.dropout(out)
        return out

class Block(nn.Module):
    def __init__(self, embed_size=512, head_num=8, sequence_length=500, dropout=0, device='cpu'):
        super().__init__()
        head_size = embed_size // head_num
        self.layer_norm1 = nn.LayerNorm(embed_size)
        self.multi_attention_heads = nn.ModuleList([
            Head(
                embed_size=embed_size, 
                head_size=head_size, 
                sequence_length=sequence_length, 
                dropout=dropout,
                device=device
            ) for _ in range(head_num)])

        self.proj = nn.Linear(embed_size, embed_size)
        
        self.layer_norm2 = nn.LayerNorm(embed_size)
        self.feedforward = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size),
            nn.ReLU(),
            nn.Linear(4 * embed_size, embed_size)
        )
        
    def forward(self, x):
        copy = x
        x = self.layer_norm1(x)
        x = torch.cat([h(x) for h in self.multi_attention_heads], dim=-1)
        x = self.proj(x)
        x += copy

        copy = x
        x = self.layer_norm2(x)
        x = self.feedforward(x)
        x += copy

        return x


class GPT(nn.Module):
    def __init__(self, vocab_size, embed_size=512, sequence_length=500, layer_number=6, head_num=8, dropout=0, device='cpu'):
        super().__init__()

        # self.token_embedding_table = TokenEmbedding(vocab_size, embed_size)
        # self.position_embedding_table = PositionalEncoding(embed_size, dropout, sequence_length)
        
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        self.position_embedding_table = nn.Embedding(sequence_length, embed_size)

        self.blocks = nn.Sequential(*[
            Block(
                embed_size=embed_size, 
                head_num=head_num,
                sequence_length=sequence_length,
                dropout=dropout,
                device=device,
            ) for _ in range(layer_number)])
        self.layer_norm = nn.LayerNorm(embed_size)
        self.linear = nn.Linear(embed_size, vocab_size)

    def forward(self, x, targets=None):
        B, T = x.shape
        token_embed = self.token_embedding_table(x)        # (B, T, C)
        posit_embed = self.position_embedding_table(x)
        x = token_embed + posit_embed

        x = self.blocks(x)
        x = self.layer_norm(x)
        logits = self.linear(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -64:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



In [11]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


model = GPT(vocab_size, embed_size=n_embd, head_num=n_head, layer_number=n_layer, dropout=dropout, device=device).to(device)
# m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

2.016065 M parameters
step 0: train loss 4.2974, val loss 4.2727
step 500: train loss 2.4060, val loss 2.4484
step 1000: train loss 2.3746, val loss 2.4482
step 1500: train loss 2.3539, val loss 2.4397
step 2000: train loss 2.3304, val loss 2.4317
step 2500: train loss 2.3137, val loss 2.4284
step 3000: train loss 2.3060, val loss 2.4263
step 3500: train loss 2.2912, val loss 2.4320
step 4000: train loss 2.2857, val loss 2.4271
step 4500: train loss 2.2779, val loss 2.4285
step 4999: train loss 2.2715, val loss 2.4215

vOFrt S: pririthury:
Tly: dit;
f oraso boo at gon irey winertire.
I r, by; s gowingevef hf tou whit s Anouryoo ce s thetouldorss cegrear' atr Warft---lerous
Thinchy wolay,

Thiche:
Hous t he MAnt cronenk lle siry, Cord hangowerast, qurn wy arerteraiat que winy;
ighes atith'scas,
Of f h as, vase serich uamige fint thethoted
Auba ccrt poamare. d;
Leancscoully, nge n heand pelouss po ge thioersaro us Touwesitr,
Thip, ly.
Thiscere, 's t, ly, s arurulf s ame:

Onou y tecoutho